In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from karateclub.dataset import GraphSetReader
from karateclub import FeatherGraph
from sklearn.model_selection import train_test_split
import networkx as nx

import itertools

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

import praw
from praw.models import MoreComments

from utils.utils import *

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
client_id = '8Ara-tL3whPJTKSzuLe0Wg'
client_secret = '5ciIZ_875ywNSWHWY7S4QcaimF788g'
user_agent = '<console:IC_proiect:1.0>'

reddit = praw.Reddit(client_id = client_id,
                    client_secret = client_secret,
                    user_agent = user_agent)

In [4]:
subredditFood = reddit.subreddit("food")
subredditHardware = reddit.subreddit("hardware")

In [16]:
def get_graphs_from_subreddit(subreddit, limit=10):
    list = []
    for submission in subreddit.hot(limit=limit):
        print(submission.title)

        G = nx.Graph()
        nodes, edges = get_graph_indices_from_submission(submission, limit=100)

        if(edges == []):
            continue
        
        G.add_edges_from(edges)
        G.add_nodes_from(nodes)

        print(G.number_of_nodes())

        list.append(G)
    return list

In [6]:
graphsFood = get_graphs_from_subreddit(subredditFood, 50)

[Homemade] bolognese
6
[homemade] Peanut Butter & Jelly Cookies
165
[homemade] Smash Burger.
15
[i ate] ice cream from an ice cream truck.
14
[I Ate] A Loaded Chef Salad.
42
[homemade] Garlic Parmesan Encrusted Pearl Onions
106
[i ate] knuckle of pork with sauerkraut and dumplings
223
[homemade] spicy pasta
25
[homemade] Cheese Pizza
8
[homemade] leftover meatloaf air fried and air fried mashed potato patty sandwich with a Kraft single and some ketchup
50


In [7]:
graphsHardware = get_graphs_from_subreddit(subredditHardware)

Reminder: Please do not submit tech support or build questions to /r/hardware
18
About Videocardz and Original Sources
84
[PR] GeIL launches world’s first DDR5 memory with dual RGB fans, EVO-V available up to DDR5-6600 64GB kits starting July
49
Radeon Preview Driver, 50 Game Benchmark [HUB]
22
NVIDIA GeForce GTX 1630 to feature 512 CUDA cores, 4GB 64-bit memory, launches May 31st - VideoCardz.com
16
Acer gaming laptop offers glasses-free 3D powered by an Nvidia RTX 3080 GPU
94
Framework Laptop upgrades to 12th gen Intel
3
Let's Examine AMD's Claim That Radeons Deliver More FPS Per Dollar Than GeForce
190
[AnandTech] The SilentiumPC Fera 5 CPU Cooler Review: Quiet Cooling on a Budget
13
Berkeley Lab: "New Silicon Nanowires Can Really Take the Heat"
6


In [8]:
model = FeatherGraph()

graphs = graphsFood + graphsHardware

# plt.figure(figsize = (10, 10))
# nx.draw_kamada_kawai(graphs[11], with_labels=True, width = .5)

model.fit(graphs)
X = model.get_embedding()

In [9]:
y = [0] * len(graphsFood) + [1] * len(graphsHardware)
y

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
y_train, y_test

([0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0], [0, 1, 1, 0])

In [11]:
downstream_model = LogisticRegression(max_iter=500, random_state=0).fit(X_train, y_train)
y_hat = downstream_model.predict_proba(X_test)[:, 1]
print(y_hat)
print(y_test)
auc = roc_auc_score(y_test, y_hat)
print('AUC: {:.4f}'.format(auc))

[0.65269624 0.57896019 0.49679595 0.30070779]
[0, 1, 1, 0]
AUC: 0.5000


In [15]:
post_url = "https://www.reddit.com/r/food/comments/ushi7j/homemade_garlic_parmesan_encrusted_pearl_onions/"
submission = reddit.submission(url = post_url)

G = nx.Graph()

nodes, edges = get_graph_indices_from_submission(submission)

G.add_edges_from(edges)
G.add_nodes_from(nodes)

model.fit([G])
X_eval = model.get_embedding()

y_pred = downstream_model.predict_proba(X_eval)
y_pred

array([[0.78543297, 0.21456703]])